In [ ]:
%matplotlib widget

In [ ]:
import matplotlib

import numpy as np
from pathlib import Path
import flammkuchen as fl
from tifffile import imread
import matplotlib.pyplot as plt 
from imaging_functions import LightsheetExperiment

from bouter.utilities import reliability 
from skimage.filters import threshold_otsu
import xarray as xr
from scipy.signal import detrend 

from stimulus_functions import stim_vel_dir_dataframe, quantize_directions
import tifffile as tiff
from scipy.signal import argrelextrema
from scipy.signal import find_peaks

In [ ]:
master_path =  Path(r"Z:\Hagar and Ot\E0040\v10\LS")
fish_list = list(master_path.glob("*f[0-9]*"))

In [ ]:
num_fish = len(fish_list)
fs = 2
color_list = ['k', 'k', 'k', 'k', 'k', 'k', 'k', 'k']
title_list = ['→', '↘', '↓', '↙', '←', '↖', '↑', '↗']

In [ ]:
regions_list = ['l_habenula_coords',
    'r_habenula_coords',
    'ipn_coords',
    'ahb_coords',
    'pretectum_coords']
region_names = ['L habenula', 'R habenula', 'IPN', 'aHB', 'Pretectum']

In [ ]:
corr_rhab =  [[],[],[],[],[],[],[],[]]
corr_lhab =  [[],[],[],[],[],[],[],[]]
corr_ipn = [[],[],[],[],[],[],[],[]]
corr_ahb =  [[],[],[],[],[],[],[],[]]
corr_pretect =  [[],[],[],[],[],[],[],[]]

In [ ]:
for fish in range(num_fish):
    path = fish_list[fish]
    traces = fl.load(path / "filtered_traces.h5", "/detr")
    sens_corr = fl.load(path / "sensory_regressors.h5", "/regressors")
    
    n_t, n_neurons = np.shape(traces)

    mean_traces = np.nanmean(traces.T, axis=0)
    mean_regs = np.nanmean(sens_corr.values, axis=0)

    traces_std = np.nanstd(traces.T, axis=0)
    regs_std = np.nanstd(sens_corr.values, axis=0)


    # Normalize the data
    traces_normalized = (traces.T - mean_traces) / traces_std
    regs_normalized = (sens_corr.values - mean_regs) / regs_std

    dot_product = np.dot(traces_normalized, regs_normalized)
    reg_corr_values = dot_product / (n_t - 1)
    
    region_masks = fl.load(path / 'regions_masks.h5')

    
    for region in range(len(region_names)):
        region_idx = region_masks[regions_list[region]]

        for direction in range(8):
            
            if region == 0:
                corr_rhab[direction] = corr_rhab[direction] + reg_corr_values[region_idx,direction].tolist()
            elif region == 1:
                corr_lhab[direction] = corr_lhab[direction] + reg_corr_values[region_idx,direction].tolist()
            elif region == 2:
                corr_ipn[direction] = corr_ipn[direction] + reg_corr_values[region_idx,direction].tolist()
            elif region == 3:
                corr_ahb[direction] = corr_ahb[direction] + reg_corr_values[region_idx,direction].tolist()
            elif region == 4:
                corr_pretect[direction] = corr_pretect[direction] + reg_corr_values[region_idx,direction].tolist()

In [ ]:
########## plot the distribution of correlation values per direction 
fig2, ax2 = plt.subplots(5,8, figsize=(12, 6), sharex=True, sharey=True)
plt.subplots_adjust(left=0.1, wspace=0.2, hspace=0.4, right=0.9)

    
for direction in range(8):
    ax2[0, direction].hist(corr_rhab[direction], bins=100, density=True, color=color_list[direction])
    ax2[1, direction].hist(corr_lhab[direction], bins=100, density=True, color=color_list[direction])
    ax2[2, direction].hist(corr_ipn[direction], bins=100, density=True, color=color_list[direction])
    ax2[3, direction].hist(corr_ahb[direction], bins=100, density=True, color=color_list[direction])
    ax2[4, direction].hist(corr_pretect[direction], bins=100, density=True, color=color_list[direction])
    ax2[0, direction].set_title(title_list[direction])
    
    for i in range(5):
        ax2[i, direction].spines['right'].set_visible(False)
        ax2[i, direction].spines['top'].set_visible(False)
        ax2[i, direction].set_xlim(-1,1)
        ax2[i, 0].set_ylabel(region_names[i])
    
        if direction > 0:
            ax2[region, direction].spines['left'].set_visible(False)

In [ ]:
file_name = "corr dist per region all fish density.jpg"
fig2.savefig(master_path / file_name, dpi=300)
file_name = "corr dist per region all fish density.pdf"
fig2.savefig(master_path / file_name, dpi=300)